machine learning translation



In [75]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

In [74]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = "/content/hin.txt"

In [76]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [77]:
input_characters

{' ',
 '!',
 '"',
 '$',
 "'",
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '€'}

In [78]:
target_characters

{'\t',
 '\n',
 ' ',
 '!',
 '"',
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 ':',
 '?',
 'A',
 'B',
 'I',
 '|',
 'ँ',
 'ं',
 'ः',
 'अ',
 'आ',
 'इ',
 'ई',
 'उ',
 'ऊ',
 'ऋ',
 'ए',
 'ऐ',
 'ऑ',
 'ओ',
 'औ',
 'क',
 'ख',
 'ग',
 'घ',
 'च',
 'छ',
 'ज',
 'झ',
 'ञ',
 'ट',
 'ठ',
 'ड',
 'ढ',
 'ण',
 'त',
 'थ',
 'द',
 'ध',
 'न',
 'प',
 'फ',
 'ब',
 'भ',
 'म',
 'य',
 'र',
 'ल',
 'व',
 'श',
 'ष',
 'स',
 'ह',
 '़',
 'ा',
 'ि',
 'ी',
 'ु',
 'ू',
 'ृ',
 'ॅ',
 'े',
 'ै',
 'ॉ',
 'ो',
 'ौ',
 '्',
 '।',
 '०',
 '१',
 '४',
 '५',
 '६',
 '७',
 '८',
 '९',
 '\u200d',
 '€'}

In [79]:
input_texts

['Wow!',
 'Duck!',
 'Duck!',
 'Help!',
 'Jump.',
 'Jump.',
 'Jump.',
 'Hello!',
 'Hello!',
 'Cheers!',
 'Cheers!',
 'Exhale.',
 'Exhale.',
 'Got it?',
 "I'm OK.",
 'Inhale.',
 'Inhale.',
 'Thanks!',
 'We won.',
 'Awesome!',
 'Come in.',
 'Get out!',
 'Go away!',
 'Goodbye!',
 'Perfect!',
 'Perfect!',
 'We lost.',
 'Welcome.',
 'Welcome.',
 'Have fun.',
 'Have fun.',
 'Have fun.',
 'I forgot.',
 'I forgot.',
 "I'll pay.",
 "I'm fine.",
 "I'm full.",
 "Let's go!",
 'Answer me.',
 'Birds fly.',
 'Excuse me.',
 'Fantastic!',
 'I fainted.',
 'I fear so.',
 'I laughed.',
 "I'm alone.",
 "I'm alone.",
 "I'm bored.",
 "I'm broke.",
 "I'm tired.",
 "It's cold.",
 'Well done!',
 'Who knows?',
 'Who knows?',
 'Who knows?',
 'Who knows?',
 'Wonderful!',
 'Birds sing.',
 'Come on in.',
 'Definitely!',
 "Don't move.",
 'Fire burns.',
 'Follow him.',
 'I can swim.',
 'I can swim.',
 'I love you.',
 'I love you.',
 'I love you.',
 'I love you.',
 'I love you.',
 "I'm coming.",
 "I'm hungry!",
 "I'm hu

In [80]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [81]:
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 2979
Number of unique input tokens: 70
Number of unique output tokens: 90
Max sequence length for inputs: 107
Max sequence length for outputs: 123


In [82]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [83]:
input_token_index

{' ': 0,
 '!': 1,
 '"': 2,
 '$': 3,
 "'": 4,
 ',': 5,
 '-': 6,
 '.': 7,
 '0': 8,
 '1': 9,
 '2': 10,
 '3': 11,
 '4': 12,
 '5': 13,
 '6': 14,
 '7': 15,
 '8': 16,
 '9': 17,
 ':': 18,
 '?': 19,
 'A': 20,
 'B': 21,
 'C': 22,
 'D': 23,
 'E': 24,
 'F': 25,
 'G': 26,
 'H': 27,
 'I': 28,
 'J': 29,
 'K': 30,
 'L': 31,
 'M': 32,
 'N': 33,
 'O': 34,
 'P': 35,
 'R': 36,
 'S': 37,
 'T': 38,
 'U': 39,
 'V': 40,
 'W': 41,
 'Y': 42,
 'a': 43,
 'b': 44,
 'c': 45,
 'd': 46,
 'e': 47,
 'f': 48,
 'g': 49,
 'h': 50,
 'i': 51,
 'j': 52,
 'k': 53,
 'l': 54,
 'm': 55,
 'n': 56,
 'o': 57,
 'p': 58,
 'q': 59,
 'r': 60,
 's': 61,
 't': 62,
 'u': 63,
 'v': 64,
 'w': 65,
 'x': 66,
 'y': 67,
 'z': 68,
 '€': 69}

In [84]:
target_token_index

{'\t': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 '"': 4,
 '(': 5,
 ')': 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 ':': 12,
 '?': 13,
 'A': 14,
 'B': 15,
 'I': 16,
 '|': 17,
 'ँ': 18,
 'ं': 19,
 'ः': 20,
 'अ': 21,
 'आ': 22,
 'इ': 23,
 'ई': 24,
 'उ': 25,
 'ऊ': 26,
 'ऋ': 27,
 'ए': 28,
 'ऐ': 29,
 'ऑ': 30,
 'ओ': 31,
 'औ': 32,
 'क': 33,
 'ख': 34,
 'ग': 35,
 'घ': 36,
 'च': 37,
 'छ': 38,
 'ज': 39,
 'झ': 40,
 'ञ': 41,
 'ट': 42,
 'ठ': 43,
 'ड': 44,
 'ढ': 45,
 'ण': 46,
 'त': 47,
 'थ': 48,
 'द': 49,
 'ध': 50,
 'न': 51,
 'प': 52,
 'फ': 53,
 'ब': 54,
 'भ': 55,
 'म': 56,
 'य': 57,
 'र': 58,
 'ल': 59,
 'व': 60,
 'श': 61,
 'ष': 62,
 'स': 63,
 'ह': 64,
 '़': 65,
 'ा': 66,
 'ि': 67,
 'ी': 68,
 'ु': 69,
 'ू': 70,
 'ृ': 71,
 'ॅ': 72,
 'े': 73,
 'ै': 74,
 'ॉ': 75,
 'ो': 76,
 'ौ': 77,
 '्': 78,
 '।': 79,
 '०': 80,
 '१': 81,
 '४': 82,
 '५': 83,
 '६': 84,
 '७': 85,
 '८': 86,
 '९': 87,
 '\u200d': 88,
 '€': 89}

In [85]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)


In [86]:
encoder_input_data

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [87]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [88]:
 encoder_input_data[0].shape

(107, 70)

In [89]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder =LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [90]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))


In [91]:
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense =Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

In [92]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [93]:
## Train the model


model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)

Epoch 1/100
38/38 [==============================] - 5s 56ms/step - loss: 1.2978 - accuracy: 0.7840 - val_loss: 2.3655 - val_accuracy: 0.6848
Epoch 2/100
38/38 [==============================] - 1s 27ms/step - loss: 1.0169 - accuracy: 0.8066 - val_loss: 1.9252 - val_accuracy: 0.6848
Epoch 3/100
38/38 [==============================] - 1s 30ms/step - loss: 0.9025 - accuracy: 0.8088 - val_loss: 1.3747 - val_accuracy: 0.6873
Epoch 4/100
38/38 [==============================] - 1s 30ms/step - loss: 0.8459 - accuracy: 0.8091 - val_loss: 1.4614 - val_accuracy: 0.6873
Epoch 5/100
38/38 [==============================] - 1s 28ms/step - loss: 0.8205 - accuracy: 0.8088 - val_loss: 1.3699 - val_accuracy: 0.6873
Epoch 6/100
38/38 [==============================] - 1s 27ms/step - loss: 0.7969 - accuracy: 0.8084 - val_loss: 1.9534 - val_accuracy: 0.6872
Epoch 7/100
38/38 [==============================] - 1s 28ms/step - loss: 0.8098 - accuracy: 0.8088 - val_loss: 1.3074 - val_accuracy: 0.6858
Epoch 

In [95]:
# Save model
model.save("s2s")


In [96]:
import tensorflow as tf


In [97]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = tf.keras.models.load_model("s2s")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = tf.keras.Model(encoder_inputs, encoder_states)

In [98]:
decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = tf.keras.Input(shape=(latent_dim,))
decoder_state_input_c = tf.keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)

In [99]:
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = tf.keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

In [100]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


In [101]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
         # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence




In [102]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

1/1 [==============================] - 0s 20ms/step
-
Input sentence: Wow!
Decoded sentence: मैं तुम्हारे पास करती हूँ।

1/1 [==============================] - 0s 20ms/step
-
Input sentence: Duck!
Decoded sentence: मैं तुम्हारे पास करती हूँ।

1/1 [==============================] - 0s 22ms/step
-
Input sentence: Duck!
Decoded sentence: मैं तुम्हारे पास करती हूँ।

1/1 [==============================] - 0s 23ms/step
-
Input sentence: Help!
Decoded sentence: मैं तुम्हारे पास करती हूँ।

1/1 [==============================] - 0s 21ms/step
-
Input sentence: Jump.
Decoded sentence: मैं तुम्हारे पास करती हूँ।

1/1 [==============================] - 0s 20ms/step
-
Input sentence: Jump.
Decoded sentence: मैं तुम्हारे पास करती हूँ।

1/1 [==============================] - 0s 23ms/step
-
Input sentence: Jump.
Decoded sentence: मैं तुम्हारे पास करती हूँ।

1/1 [==============================] - 0s 24ms/step
-
Input sentence: Hello!
Decoded sentence: मैं तुम्हारे पास करती हूँ।

1/1 [===================